In [1]:
import scipy.stats as sts

In [2]:
def distribution(rho: float, population_size: int):
    return sts.norm(rho, ((1 + rho**2) / population_size) ** 0.5)

In [36]:
def generate_population(rho: float, size: int):
    p_value_estimator = lambda statistic: distribution(0, size).sf(statistic) *2 
    dist = sts.multivariate_normal([0, 0], [[1, rho], [rho, 1]])
    for _ in range(500):
        samples = dist.rvs(size=size)
        yield p_value_estimator(abs((samples[:, 0] @ samples[:, 1]) / size))

In [37]:
p_values_spread = [
    experiment_p_value for experiment_p_value in generate_population(0, 100)
] + [experiment_p_value for experiment_p_value in generate_population(0.2, 100)]

In [39]:
sum([p < 0.05 for p in p_values_spread[:500]]) / 500, sum([p < 0.05 for p in p_values_spread[500:]]) / 500

(np.float64(0.044), np.float64(0.49))

Bonferroni

In [40]:
sum([p < 0.05/ 500 for p in p_values_spread[:500]])

np.int64(0)

In [ ]:
def sampler(distribution: rv_continuous, sample_size: int, number_of_populations: int):
    for _ in number_of_populations:
        yield distribution.rvs(sample_size)

In [ ]:
for sample in sampler(norm(0.5), 100, 500):
    sample